# Data Enrichment

Data enrichment and data aggregation are the processes involved around joining merging datasets, creating new columns, calculating values on certain windows, grouping into bins, or even changing the values. All of these processes will assist in the data analysis by providing specific insight into the data. Things such as severity of anomalous data, rolling averages, cumulative sums, or quantities grouped by ages can all make the data easier to interpret. We'll focus on the following for this class:
- Querying and Merging dataframes
- Aggregating Dataframes

In [ ]:
import pandas as pd

## Querying data frames
In previous sections we talked about how we learned about how to filter a data frame by building some sort of mapping that would indicate exactly which rows to keep, and which to remove. Another approach is to use the `query` method. 

Below we import a more expanded weather data set that we've seen in previous lessons. 

In [ ]:
weather = pd.read_csv('https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_04/data/nyc_weather_2018.csv')
weather

We take note of the `datatype` column. Previously, we had seen only temperature related data, however, now we see a few more options. 

In [ ]:
weather.datatype.unique()

We could use the same method discussed earlier to create a map or filter as follows:

In [ ]:
snow_data_mask = weather[
    (weather.datatype == 'SNOW') & (weather.value > 0)
]
snow_data_mask.head()

However, we do have a method called `query` which allows us to write our filters in a format closer to that commonly seen in SQL or even python boolean logic.

In [ ]:
snow_data = weather.query('datatype == "SNOW" and value > 0')
snow_data.head()

We can even demonstrate that both of these methods return the exact same dataframe

In [ ]:
snow_data_mask.equals(snow_data)

The method that you choose largely depends on preference. Apparently there are some potential speed differences complexity differences that could be encountered, however, that is well beyond the scope of this course. 

## Merging DataFrames

Another crucial part of data analysis is combining datasets together, creating a more complete understanding of the data. There are two types of merges that we typically talk about. Using the vernacular common in from SQL: 
- `join`: A join combines columns of one dataset to the columns of another dataset based on some condition on the row. If the condition is meet, the columns from dataset B are attached to dataset A. 
- `union`: A Union can be thought of as appending two data sets that share the same columns.

Join is the more complex between the two, so we'll mostly talk about that.

### Joining Datasets
So far in this class, we have only worked with a single dataset. Joins provide us the ability to take two separate tables or dataframes with related information, and combine them into a single table. For the weather data we've been using, we might perform a join to attach a physical location to the weather measurements using the weather station's id to gain a better idea of how the weather patterns are distributed.

The most common types of joins are described below using venn diagrams.  

![joins diagram](Assets/joins.jpg "Joins Diagram")

Think of the circles as the complete set of rows for each dataframe and the shaded region as the rows that are returned as a result of the join. Described briefly, 
- Inner joins return only the rows that are present in both dataframes,  
- Left (Right) joins return all of the rows from the left (right) dataframe, leaving all the values as null or missing (depending on your language) for the columns from the right (left) table if there was no matching ID found in that table
- Full or Outer joins return all rows from both tables, leaving missing values in the columns on both sides for missing IDs in either table (Think the left and right joins happening at the same time)

Since it can be easier to understand something by doing, let's join some information about the weather stations to the rows of observations from above.

In [ ]:
# reading in the weather station data
stations = pd.read_csv("https://raw.githubusercontent.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/master/ch_04/data/weather_stations.csv")
stations.head()

The first thing we need to do to join two dataframes is determine the rows that we can join on. Note the row called `id`. This value is designed as a unique identifier for each row. If we look at the data from the `weather` dataframe we imported above, we could find that the column `station` contains data that seems to match `stations.id`

In [ ]:
weather.head()

In order to determine how to join, we can take a look at the shape of each dataframe. It is also specifically important to check the number of unique values of the column we want to join on. We can check that below

In [ ]:
print(f"Weather: {weather.shape}")
print(f"Stations: {stations.shape}")
print(f"Unique stations from Weather: {weather.station.unique().shape}")
print(f"Unique stations from Stations: {stations.id.unique().shape}")

Here we can see that `weather` has more rows than `stations`. In order to preserve the actual data, we want to make sure that we are merging the information from `weather` to `stations`. This is due to the way relation databases store their information, which unfortunately is beyond the scope of this course. We do see that we will lose data on some of the stations (note that `stations.id` has more values than `weather.station`), however this is acceptable since our primary focus is the weather data, not the station data. We could probably go a step further and double check that all of station IDs in `weather` are also present in `stations`, but I'm not that concerned about that right now. 

All of the joins in we are interested in this course can be executed using the `merge` method and specifying keyword arguments, however there are other methods that could be called instead.

Inner join is the default join type of `merge`. The most important argument is the second dataframe to be merged. On top of that you may need to specify the names of the columns to be checked.

In [ ]:
inner_joined_tables = weather.merge(stations, left_on='station', right_on='id')
inner_joined_tables.head()

Note that now we have all of the weather data from before (datatype, date, value, etc.) on the left side of the table, and all of the location data on the right (latitude, longitude, and elevation). Unfortunitely we do have two columns with identical informaiton, but this could be solved easiy by dropping one of the columns after the fact or by renaming the one column beforehand. If the joining columns have the same name, python only includes one column with that name. 

In [ ]:
weather.merge(stations.rename({'id':'station'}, axis='columns').drop('elevation', axis='columns'), on='station').head()

Note that with this inner join, there are no missing values on either side of the table (at least none that isn't just bad or missing data)

In [ ]:
inner_joined_tables.query('id.isna() or station.isna()').shape

This is where an inner join differs from right, left, and outer joins. With these types of joins, we allow for the possibility of missing data. Let's attempt a left join. 

In [ ]:
left_joined_tables = weather.merge(stations, left_on='station', right_on='id', how='left')
left_joined_tables.query("id.isna() or station.isna()").shape

In this particular case, we don't have any missing data `station`. This tells us that all station ids found in `weather` were part of the `stations` dataframe. However, remeber that `stations` has more unique values in its `id` column than `weather.station`. If we perform a right join, we know for sure that we will get some missing weather data, because `weather.station` does not have all the values that `stations.id` has. 

In [ ]:
right_joined_tables = weather.merge(stations, left_on='station', right_on='id', how='right')
right_joined_tables.query("id.isna() or station.isna()")

Notice now that we have a set of rows for which the weather station data is present, but under the weather data, we have a bunch of missing values. This indicates to us that the some of the weather stations present in the `stations` dataframe are *not* present in the `weather` dataframe. Doing a quick bit of math, we can confirm that the missing rows matches up with what we expected. Performing an outer join is essentially the same as a left and a right join together, with the above behavior expected for both the left and right tables. 

In [ ]:
len(stations.id.unique()) - len(weather.station.unique())

The only difference between an outer join and a right or left join is that we are essentially doing both a left and a right join at the same time. We would get all rows where `weather.station` had a matching value in `stations.id`, all rows where `weather.station` had a value that was not present in `station.id` (in this case that number is 0), and all rows where `station.id` has a value not present in `weather.station`.

## Aggregation
Another important function of data analysis is the idea of aggregation. This referes to the process of taking the data and rolling it up into a single value or set of values instead of looking at the individual measurements. We might want the summed, average, or maximum value of the dataset. We can use fairly simple function calls that we have acctually used to a degree before. I will use pivot tables to combine the different aggregations together in our weather data example. 

In [ ]:
weather.date = pd.to_datetime(weather.date)
weather_pivot = weather.set_index(['date', 'station'])\
    .pivot(columns='datatype', values='value')[['PRCP', 'SNOW', 'TAVG', 'TMAX', 'TMIN']]
weather_pivot.query('station == "GHCND:USW00094728"')[['PRCP', 'SNOW']].sum()


In the above example, we filter pivot the dataframe so that each datatype is its own column, then we can filter on values from only one specific station, and only select the columns where it makes sense for the aggregation we are performing. In doing this, we have computed a value that reports the total amount of rain and snow fall for the entire measurement period. 

The above gives us the totals over the entire dataset. We could make this perform the same logic to calculate things like the average per day, or find the days with the most or the least. The method calls are fairly intuitive, and can be quickly found in Panda's documentation. A functionality very commonly used with these aggregation methods is that of grouping (called by `groupby`) to partition the data and perform the aggregation function on each group.  

In [ ]:
weather_pivot.groupby("station")[['PRCP', 'SNOW']].sum()

We could find the same in or first example by grouping, then filtering at the end.

In [ ]:
weather_pivot.groupby("station")[['PRCP', 'SNOW']].sum().query('station == "GHCND:USW00094728"')

Instead of grouping by the station, we could instead group by the date to find daily statistics such as the minimum of each column on every day across all stations. 

In [ ]:
weather_pivot.groupby('date').min()

Additionally, you can group by multiple columns. It will create a set of nested groups that can be aggregated together for further separation of the metrics being calculated. 

There is also a pandas class called `Grouper` that seems to allow for even more complex grouping functionality, but we will not go into that class in this lesson.

## Window Functions

Window functions are a very interesting and useful function that can provide some insight. Sometimes, we want to know the maximum or the average of an entire column, but it can be just as interesting to know the rolling average of some sort of data. Window functions allow us to perform calculations on a group of rows that are close to each other in some way. Using the weather data, we can calculate the average rainfall over the past week. Using the method `rolling`, we can include a predefined set of values in our calculation. In this example, we compute the rolling average of rainfall over the last 7 days at a specific station. 

In [ ]:
rainfall = weather.query("datatype == 'PRCP' and station == 'GHCND:USW00094728'")\
    .set_index("date")\
    .assign(rolling_average=lambda x: x.value.rolling(7).mean())

In [ ]:
rainfall

Note that the first few rows do not have a value, as rolling does not have enough rows to pull from at this point. 